In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import plotly.graph_objects as go
import altair as alt

# Read ICE arrests data from GitHub
url = "https://github.com/deportationdata/ice/raw/refs/heads/main/data/arrests-latest.xlsx"

# Download the file with requests
response = requests.get(url)
df = pd.read_excel(BytesIO(response.content))

# Display basic information about the data
print(f"Data shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

Data shape: (291722, 23)

Column names:
['apprehension_date', 'apprehension_state', 'apprehension_aor', 'final_program', 'final_program_group', 'apprehension_method', 'apprehension_criminality', 'case_status', 'case_category', 'departed_date', 'departure_country', 'final_order_yes_no', 'final_order_date', 'birth_year', 'citizenship_country', 'gender', 'apprehension_site_landmark', 'unique_identifier', 'apprehension_date_time', 'duplicate_likely', 'file_original', 'sheet_original', 'row_original']

First few rows:


,apprehension_date,apprehension_state,apprehension_aor,final_program,final_program_group,apprehension_method,apprehension_criminality,case_status,case_category,departed_date,...,birth_year,citizenship_country,gender,apprehension_site_landmark,unique_identifier,apprehension_date_time,duplicate_likely,file_original,sheet_original,row_original
0,2023-09-01,CALIFORNIA,San Francisco Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Federal Incarceration,1 Convicted Criminal,6-Deported/Removed - Deportability,[16] Reinstated Final Order,2023-09-02,...,1972,MEXICO,Male,"FRE GENERAL AREA, NON-SPECIFIC",3ddc9dfa23c14851a7cd709ad2e6c52a4e36a08b,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,69540
1,2023-09-01,SOUTH CAROLINA,Atlanta Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Local Incarceration,2 Pending Criminal Charges,8-Excluded/Removed - Inadmissibility,[8C] Excludable / Inadmissible - Administrativ...,2024-01-17,...,1994,HONDURAS,Male,"RICHLAND COUNTY, SC",8b3087d9852e9db2203541ebb2fc90826c74a647,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,156756
2,2023-09-01,NaN,NaN,Alternatives to Detention,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,E-Charging Document Canceled by ICE,[8A] Excludable / Inadmissible - Hearing Not C...,NaT,...,1966,ECUADOR,Male,NaN,db5178743c5753e8acf6f2f2eca88af68395073f,2023-09-01 00:19:00,1.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,247560
3,2023-09-01,NaN,NaN,Alternatives to Detention,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,ACTIVE,[8D] Excludable / Inadmissible - Under Adjudic...,NaT,...,1966,ECUADOR,Male,NaN,db5178743c5753e8acf6f2f2eca88af68395073f,2023-09-01 00:19:00,1.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,247561
4,2023-09-01,NaN,Phoenix Area of Responsibility,Detained Docket Control,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,ACTIVE,[8B] Excludable / Inadmissible - Under Adjudic...,NaT,...,1999,INDIA,Female,NaN,d24c268740238eff4206067e9be369a2b872bc51,2023-09-01 01:09:44,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,237391


In [2]:
# Check data structure to understand columns
print("Columns:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nSample data:")
print(df.head(10))

Columns: ['apprehension_date', 'apprehension_state', 'apprehension_aor', 'final_program', 'final_program_group', 'apprehension_method', 'apprehension_criminality', 'case_status', 'case_category', 'departed_date', 'departure_country', 'final_order_yes_no', 'final_order_date', 'birth_year', 'citizenship_country', 'gender', 'apprehension_site_landmark', 'unique_identifier', 'apprehension_date_time', 'duplicate_likely', 'file_original', 'sheet_original', 'row_original']

Data types:
apprehension_date             datetime64[ns]
apprehension_state                    object
apprehension_aor                      object
final_program                         object
final_program_group                   object
apprehension_method                   object
apprehension_criminality              object
case_status                           object
case_category                         object
departed_date                 datetime64[ns]
departure_country                     object
final_order_yes_no   

In [3]:
# Check exact column names
print("Exact columns:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())

Exact columns: ['apprehension_date', 'apprehension_state', 'apprehension_aor', 'final_program', 'final_program_group', 'apprehension_method', 'apprehension_criminality', 'case_status', 'case_category', 'departed_date', 'departure_country', 'final_order_yes_no', 'final_order_date', 'birth_year', 'citizenship_country', 'gender', 'apprehension_site_landmark', 'unique_identifier', 'apprehension_date_time', 'duplicate_likely', 'file_original', 'sheet_original', 'row_original']

First 5 rows:
  apprehension_date apprehension_state                      apprehension_aor  \
0        2023-09-01         CALIFORNIA  San Francisco Area of Responsibility   
1        2023-09-01     SOUTH CAROLINA        Atlanta Area of Responsibility   
2        2023-09-01                NaN                                   NaN   
3        2023-09-01                NaN                                   NaN   
4        2023-09-01                NaN        Phoenix Area of Responsibility   

                final_progr

In [4]:
# Let me check if monthly_arrests variable exists from previous work
if 'monthly_arrests' in dir():
    print("monthly_arrests exists!")
    print(monthly_arrests.head(20))
    print("\nColumns:", monthly_arrests.columns.tolist())
else:
    print("monthly_arrests doesn't exist, using df")
    print("df columns:", df.columns.tolist())
    print("\ndf shape:", df.shape)
    print("\ndf head:")
    print(df.head())

monthly_arrests doesn't exist, using df
df columns: ['apprehension_date', 'apprehension_state', 'apprehension_aor', 'final_program', 'final_program_group', 'apprehension_method', 'apprehension_criminality', 'case_status', 'case_category', 'departed_date', 'departure_country', 'final_order_yes_no', 'final_order_date', 'birth_year', 'citizenship_country', 'gender', 'apprehension_site_landmark', 'unique_identifier', 'apprehension_date_time', 'duplicate_likely', 'file_original', 'sheet_original', 'row_original']

df shape: (291722, 23)

df head:
  apprehension_date apprehension_state                      apprehension_aor  \
0        2023-09-01         CALIFORNIA  San Francisco Area of Responsibility   
1        2023-09-01     SOUTH CAROLINA        Atlanta Area of Responsibility   
2        2023-09-01                NaN                                   NaN   
3        2023-09-01                NaN                                   NaN   
4        2023-09-01                NaN        Phoeni

In [66]:
# Prepare data for Vega-Lite with proper formatting
chart_data = monthly_arrests.copy()
chart_data['date'] = pd.to_datetime(chart_data['year_month'])
chart_data['year'] = chart_data['date'].dt.year
chart_data['month_num'] = chart_data['date'].dt.month
chart_data['month_letter'] = chart_data['date'].dt.strftime('%b').str[0]  # First letter: S, O, N, etc.
chart_data['admin'] = chart_data['date'].apply(lambda x: 'Trump' if x >= pd.Timestamp('2025-01-01') else 'Biden')
chart_data['color'] = chart_data['admin'].map({'Biden': '#808080', 'Trump': '#d62728'})

# Create x-axis position (sequential index)
chart_data['x_pos'] = range(len(chart_data))

print("Chart data ready:")
print(chart_data[['date', 'month_letter', 'year', 'arrests_thousands', 'admin']].to_string())

Chart data ready:
         date month_letter  year  arrests_thousands  admin
0  2023-09-01            S  2023             10.515  Biden
1  2023-10-01            O  2023             10.004  Biden
2  2023-11-01            N  2023              8.825  Biden
3  2023-12-01            D  2023              9.807  Biden
4  2024-01-01            J  2024              8.462  Biden
5  2024-02-01            F  2024              9.540  Biden
6  2024-03-01            M  2024              9.507  Biden
7  2024-04-01            A  2024             10.136  Biden
8  2024-05-01            M  2024             10.223  Biden
9  2024-06-01            J  2024              8.459  Biden
10 2024-07-01            J  2024              9.442  Biden
11 2024-08-01            A  2024              9.141  Biden
12 2024-09-01            S  2024              8.407  Biden
13 2024-10-01            O  2024              9.580  Biden
14 2024-11-01            N  2024              8.246  Biden
15 2024-12-01            D  2024      

In [90]:
import json

# Create Vega-Lite specification matching the Economist-style chart
spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v6.json",
    "config": {
        "axis": {
            "labelFontSize": 12,
            "titleFontSize": 13,
            "labelFont": "Arial, sans-serif",
            "titleFont": "Arial, sans-serif"
        },
        "title": {
            "fontSize": 18,
            "fontWeight": "bold",
            "font": "Arial, sans-serif",
            "anchor": "start"
        },
        "view": {
            "stroke": None
        }
    },
    "width": 800,
    "height": 500,
    "title": {
        "text": "Migrant arrests by Immigration and",
        "subtitle": "Customs Enforcement (ICE), '000",
        "anchor": "start"
    },
    "data": {"values": chart_data.to_dict('records')},
    "layer": [
        {
            # Vertical separator line - placed first so bars appear on top
            "data": {
                "values": [{"x": 16}]
            },
            "mark": {
                "type": "rule",
                "strokeWidth": 2,
                "color": "black"
            },
            "encoding": {
                "x": {"field": "x", "type": "quantitative", "axis": None},
                "y": {"datum": 0},
                "y2": {"datum": 30}
            }
        },
        {
            "mark": {
                "type": "bar",
                "tooltip": True,
                "color": "#d62728",
                "width": {"band": 0.9}
            },
            "encoding": {
                "x": {
                    "field": "x_pos",
                    "type": "quantitative",
                    "axis": {
                        "title": None,
                        "labels": False,
                        "ticks": False,
                        "grid": False,
                        "domain": True,
                        "domainColor": "#000000"
                    },
                    "scale": {"domain": [-0.5, 22.5]}
                },
                "y": {
                    "field": "arrests_thousands",
                    "type": "quantitative",
                    "axis": {
                        "title": None,
                        "grid": True,
                        "gridColor": "#e0e0e0",
                        "tickCount": 7,
                        "labels": True,
                        "domain": False,
                        "ticks": False
                    },
                    "scale": {"domain": [0, 30]}
                },
                "tooltip": [
                    {"field": "date", "type": "temporal", "title": "Month", "format": "%b %Y"},
                    {"field": "arrests_thousands", "type": "quantitative", "title": "Arrests (thousands)", "format": ".1f"}
                ]
            }
        },
        {
            # Short tick marks for regular month boundaries
            "data": {
                "values": [
                    {"x": -0.5}, {"x": 0.5}, {"x": 1.5}, {"x": 2.5}, {"x": 3.5}, 
                    {"x": 4.5}, {"x": 5.5}, {"x": 6.5}, {"x": 7.5}, {"x": 8.5}, 
                    {"x": 9.5}, {"x": 10.5}, {"x": 11.5}, {"x": 12.5}, {"x": 13.5}, 
                    {"x": 14.5}, {"x": 15.5}, {"x": 16.5}, {"x": 17.5}, {"x": 18.5}, 
                    {"x": 19.5}, {"x": 20.5}, {"x": 21.5}, {"x": 22.5}
                ]
            },
            "mark": {
                "type": "rule",
                "strokeWidth": 1,
                "color": "#000000"
            },
            "encoding": {
                "x": {"field": "x", "type": "quantitative", "axis": None},
                "y": {"datum": 0},
                "y2": {"datum": -0.30}
            }
        },
        {
            # Long tick marks for year boundaries (at x = 3.5 for 2023->2024, and x = 15.5 for 2024->2025)
            "data": {
                "values": [
                    {"x": 3.5},  # Between Dec 2023 and Jan 2024
                    {"x": 15.5}  # Between Dec 2024 and Jan 2025
                ]
            },
            "mark": {
                "type": "rule",
                "strokeWidth": 1,
                "color": "#000000"
            },
            "encoding": {
                "x": {"field": "x", "type": "quantitative", "axis": None},
                "y": {"datum": 0},
                "y2": {"datum": -0.60}
            }
        },
        {
            "mark": {
                "type": "text",
                "align": "center",
                "baseline": "top",
                "dy": 5,
                "fontSize": 11,
                "fontWeight": "normal"
            },
            "encoding": {
                "x": {"field": "x_pos", "type": "quantitative", "axis": None},
                "y": {"datum": 0},
                "text": {"field": "month_letter", "type": "nominal"}
            }
        },
        {
            "data": {
                "values": [
                    {"year": "2023", "x_pos": 1.5},
                    {"year": "2024", "x_pos": 10},
                    {"year": "2025", "x_pos": 19}
                ]
            },
            "mark": {
                "type": "text",
                "fontSize": 13,
                "fontWeight": "bold",
                "baseline": "top",
                "dy": 25
            },
            "encoding": {
                "x": {"field": "x_pos", "type": "quantitative", "axis": None},
                "y": {"datum": 0},
                "text": {"field": "year", "type": "nominal"}
            }
        },
        {
            "data": {
                "values": [
                    {"label": "Biden", "x_pos": 8},
                    {"label": "Trump", "x_pos": 19}
                ]
            },
            "mark": {
                "type": "text",
                "fontSize": 14,
                "fontWeight": "bold",
                "baseline": "bottom",
                "dy": -10
            },
            "encoding": {
                "x": {"field": "x_pos", "type": "quantitative", "axis": None},
                "y": {"datum": 32},
                "text": {"field": "label", "type": "nominal"}
            }
        }
    ]
}

# Display the chart
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [85]:
# Save using Altair's built-in method
output_file = '../graphs/ice_migrant_arrests_economist.json'

chart = alt.Chart.from_dict(spec)
chart.save(output_file)

print(f"Chart saved to: {output_file}")

Chart saved to: ../graphs/ice_migrant_arrests_economist.json
